In [1]:
import pandas as pd
import numpy as np

file_furinture_sales_label=pd.ExcelFile("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/BL_output_store_type_furniture_3_months_JL_2020-01-16.xlsx")
df_store_furniture_label=file_furinture_sales_label.parse('store_type_recent_3_months',dtype=str)
df_store_furniture_label=df_store_furniture_label[['location_id','store_furniture_group']]
df_store_furniture_label['store_furniture_group']="Furniture"+df_store_furniture_label['store_furniture_group']
print(df_store_furniture_label.shape,df_store_furniture_label['location_id'].nunique())

#######

df_store_competition=pd.read_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/Competitor_Label/BL_competitor_count_JL_2020-01-21.csv",dtype=str)
del df_store_competition['competitor_count']
df_store_competition['competition_group']="Competitors"+df_store_competition['competition_group']
print(df_store_competition.shape,df_store_competition['location_id'].nunique())

(1426, 2) 1426
(1438, 2) 1438


In [2]:
store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20200101-135601-098.txt",
                        dtype=str,sep="|",usecols=['location_id'])
store_list=store_list.drop_duplicates()
store_list=store_list[store_list['location_id']!="145"]
store_list=store_list[store_list['location_id']!="6990"]


In [3]:
TA_excel=pd.ExcelFile("/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Planner_Request/Ramiro_TA_request_20200124/BL_zips_in_TA_2020_updated_30_new_stores_JL_2020-01-27.xlsx")
print(TA_excel.sheet_names)

df_zips_by_store=TA_excel.parse("view_by_store_and_zip",dtype=str)
df_zips_by_store=df_zips_by_store[['location_id','zip_cd','zip_type']]

df_store_list_with_zip=pd.merge(store_list,df_zips_by_store,on="location_id",how="left")
df_store_list_with_zip.shape

['unique_zips', 'view_by_TA', 'view_by_store', 'view_by_store_and_zip', 'store_list_full_202001']


(23634, 3)

In [4]:
df_store_list_with_zip=pd.merge(df_store_list_with_zip,df_store_furniture_label,on="location_id",how="left")
df_store_list_with_zip=pd.merge(df_store_list_with_zip,df_store_competition,on="location_id",how="left")
df_store_list_with_zip['zip_cd'].nunique()

13324

In [5]:
df_store_list_with_zip.head(6)

,location_id,zip_cd,zip_type,store_furniture_group,competition_group
0,1,43227,trans_P,FurnitureHigh,CompetitorsLow/No
1,1,43207,trans_P,FurnitureHigh,CompetitorsLow/No
2,1,43209,trans_P,FurnitureHigh,CompetitorsLow/No
3,1,43232,trans_P,FurnitureHigh,CompetitorsLow/No
4,1,43125,trans_S,FurnitureHigh,CompetitorsLow/No
5,1,43205,trans_S,FurnitureHigh,CompetitorsLow/No


In [6]:
# writer=pd.ExcelWriter("./BL_zips_by_Furniture_and_Competitors_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
len_max=0
dict_list_zip={}
for furniture_seg, df_group in df_store_list_with_zip.groupby("store_furniture_group"):
    df=df_group['zip_cd'].drop_duplicates()
    df=df.rename(columns={"zip_cd":furniture_seg})
    print(furniture_seg,df.shape)
    if len(df)>len_max:
        len_max=len(df)
    dict_list_zip.update({furniture_seg:df[furniture_seg].unique().tolist()})
    # df.to_excel(writer,furniture_seg,index=False)
    
for competition_seg, df_group in df_store_list_with_zip.groupby("competition_group"):
    df=df_group['zip_cd'].drop_duplicates()
    df=df.rename(columns={"zip_cd":competition_seg})
    print(competition_seg,df.shape)
    if len(df)>len_max:
        len_max=len(df)
    dict_list_zip.update({competition_seg:df[competition_seg].unique().tolist()})
    # df.to_excel(writer,furniture_seg,index=False)

for k,v in  dict_list_zip:
    v.sort()
    new_v=v+[np.nan]*(len_max-len(v))
    dict_list_zip.update({k:new_v})
    
df_unique_all_zips=pd.DataFrame(dict_list_zip)
df_unique_all_zips.to_excel(writer,"all_zips_by_seg",index=False)
# writer.save()

FurnitureHigh (11024,)
FurnitureLow (4031,)
CompetitorsHigh (8853,)
CompetitorsLow/No (6159,)


In [28]:
df_store_list_with_zip.head(2)

,location_id,zip_cd,zip_type,store_furniture_group,competitor_count,competition_group
0,1,43227,trans_P,FurnitureHigh,49,Low/No
1,1,43207,trans_P,FurnitureHigh,49,Low/No


13324